In [2]:
from agential.cog.prompts.critic import CRITIC_INSTRUCTION_HUMANEVAL
from agential.cog.functional.critic import _prompt_agent

import dotenv

from langchain_community.chat_models.openai import ChatOpenAI

llm = ChatOpenAI()

In [ ]:
question = """from typing import List 

def has_close_elements(numbers: List[float], threshold: float) -> bool: 
    \"\"\"Check if in given list of numbers, are any two numbers closer to each other than given threshold. 
    >>> has_close_elements([1.0, 2.0, 3.0], 0.5) False 
    >>> has_close_elements([1.0, 2.8, 3.0, 4.0, 5.0, 2.0], 0.3)
    True
    \"\"\"
"""

out = _prompt_agent(
    llm=llm,
    question=question,
    examples=examples,
    prompt=prompt
)